# Day 6: Anomaly Detection for Fraud

**Unsupervised Fraud Detection Using Isolation Forest**

## Overview
- **Objective**: Detect fraud without labeled data
- **Method**: Isolation Forest (unsupervised anomaly detection)
- **Advantage**: Works with NO fraud labels needed during training

## What You'll Learn
1. **Isolation Forest**: How it isolates anomalies
2. **Training**: Train on legitimate data only
3. **Detection**: Score new transactions for anomaly
4. **Ensemble Methods**: Combine multiple detectors

---

## 1. Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported!")

## 2. Understanding Isolation Forest

**How It Works:**

1. **Random Partitioning**: Recursively split data with random thresholds
2. **Path Length**: Anomalies are isolated quickly (short paths)
3. **Scoring**: Shorter path = more anomalous

**Key Insight**: Anomalies are "few and different", so they're easier to isolate!

In [ ]:
# Visualize how Isolation Forest works
np.random.seed(42)

# Generate 2D data
normal = np.random.randn(200, 2) * 1.5
anomalies = np.array([[3, 3], [-3, 3], [3, -3], [-4, -2], [5, 0]])

X = np.vstack([normal, anomalies])
y = np.array([0]*200 + [1]*5)

# Fit Isolation Forest
clf = IsolationForest(contamination=0.02, random_state=42)
clf.fit(X)

# Predict
y_pred = clf.predict(X)
anomaly_scores = -clf.score_samples(X)  # Convert to positive scores

# Plot
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(normal[:, 0], normal[:, 1], c='blue', alpha=0.5, label='Normal')
plt.scatter(anomalies[:, 0], anomalies[:, 1], c='red', s=100, label='Anomalies', edgecolors='black')
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('True Labels', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
scatter = plt.scatter(X[:, 0], X[:, 1], c=anomaly_scores, cmap='YlOrRd', alpha=0.7)
plt.colorbar(scatter, label='Anomaly Score')
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.title('Isolation Forest Anomaly Scores', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Isolation Forest Results:")
print(f"  Detected {np.sum(y_pred == -1)} anomalies")
print(f"  Actual anomalies: {np.sum(y == 1)}")

## 3. Generate Fraud Detection Data

In [ ]:
# Generate transaction data
np.random.seed(42)

# Normal transactions
n_normal = 5000
normal_transactions = {
    'amount': np.random.lognormal(3, 0.8, n_normal).clip(0, 5000),
    'hour': np.random.randint(0, 24, n_normal),
    'day_of_week': np.random.randint(0, 7, n_normal),
    'merchant_category': np.random.randint(0, 10, n_normal),
}

# Fraud transactions (anomalous behavior)
n_fraud = 100
fraud_transactions = {
    'amount': np.random.lognormal(5, 0.5, n_fraud).clip(1000, 15000),  # Higher amounts
    'hour': np.random.choice([0, 1, 2, 3, 4, 22, 23], n_fraud),  # Unusual hours
    'day_of_week': np.random.randint(0, 7, n_fraud),
    'merchant_category': np.random.randint(0, 10, n_fraud),
}

# Combine
df = pd.DataFrame({**{k: np.concatenate([v, fraud_transactions[k]]) for k, v in normal_transactions.items()}})
df['is_fraud'] = [0]*n_normal + [1]*n_fraud

# Shuffle
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Dataset shape: {df.shape}")
print(f"Fraud rate: {df['is_fraud'].mean()*100:.2f}%")
print(f"\nSample data:")
print(df.head())

## 4. Train Isolation Forest (Legitimate Data Only)

In [ ]:
# IMPORTANT: Train only on legitimate data (Class=0)
legitimate_data = df[df['is_fraud'] == 0].drop('is_fraud', axis=1)

# Fit Isolation Forest
iso_forest = IsolationForest(
    n_estimators=100,
    max_samples='auto',
    contamination=0.02,  # Expected anomaly rate
    random_state=42,
    n_jobs=-1
)

iso_forest.fit(legitimate_data)

print("✅ Isolation Forest trained on legitimate data only!")
print(f"   Trained on {len(legitimate_data)} legitimate transactions")

## 5. Detect Anomalies

In [ ]:
# Get anomaly scores for all data
X_all = df.drop('is_fraud', axis=1)

# Isolation Forest returns negative scores (lower = more anomalous)
# Convert to positive scores (higher = more anomalous)
anomaly_scores = -iso_forest.score_samples(X_all)

# Binary predictions
predictions = iso_forest.predict(X_all)
predictions_binary = (predictions == -1).astype(int)  # -1 = anomaly

print("Anomaly Detection Results:")
print(f"  Total transactions: {len(df)}")
print(f"  Detected as anomalous: {np.sum(predictions_binary)}")
print(f"  Actual fraud cases: {df['is_fraud'].sum()}")
print(f"  Fraud detection rate: {np.sum(predictions_binary[df['is_fraud'] == 1]) / df['is_fraud'].sum() * 100:.1f}%")

## 6. Evaluate Performance

In [ ]:
# Calculate metrics
y_true = df['is_fraud'].values
y_pred_binary = predictions_binary

auc = roc_auc_score(y_true, anomaly_scores)

print("="*60)
print("ISOLATION FOREST - EVALUATION")
print("="*60)
print(f"\nAUC-ROC: {auc:.3f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred_binary, 
                          target_names=['Legitimate', 'Fraud']))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_binary)
print("\nConfusion Matrix:")
print(cm)
print(f"\nFraud Recall: {cm[1,1] / (cm[1,0] + cm[1,1]) * 100:.1f}%")

## 7. Optimize Threshold

In [ ]:
from sklearn.metrics import f1_score, precision_recall_curve

# Find optimal threshold
thresholds = np.percentile(anomaly_scores, np.arange(95, 100, 0.5))
f1_scores = []

for threshold in thresholds:
    preds = (anomaly_scores >= threshold).astype(int)
    f1 = f1_score(y_true, preds)
    f1_scores.append(f1)

best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
best_f1 = f1_scores[best_idx]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(thresholds, f1_scores, 'o-', linewidth=2, markersize=4)
plt.axvline(x=best_threshold, color='r', linestyle='--', label=f'Best threshold = {best_threshold:.2f}')
plt.xlabel('Anomaly Score Threshold', fontsize=12)
plt.ylabel('F1 Score', fontsize=12)
plt.title('F1 Score vs Threshold', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Apply optimal threshold
optimal_preds = (anomaly_scores >= best_threshold).astype(int)

print(f"Optimal threshold: {best_threshold:.3f}")
print(f"Optimal F1-Score: {best_f1:.3f}")
print(f"Fraud recall at optimal: {confusion_matrix(y_true, optimal_preds)[1,1] / (confusion_matrix(y_true, optimal_preds)[1,0] + confusion_matrix(y_true, optimal_preds)[1,1]) * 100:.1f}%")

## 8. Ensemble of Anomaly Detectors

In [ ]:
# Scale data for SVM
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_all)

# Train multiple detectors
print("Training ensemble of anomaly detectors...")

# 1. Isolation Forest
iso_forest = IsolationForest(contamination=0.02, random_state=42)
iso_forest.fit(legitimate_data)
iso_scores = -iso_forest.score_samples(X_all)

# 2. One-Class SVM (trained on legitimate data)
oc_svm = OneClassSVM(nu=0.02, kernel='rbf')
oc_svm.fit(scaler.transform(legitimate_data))
svm_scores = -oc_svm.score_samples(X_scaled)

# 3. Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.02, novelty=True)
lof.fit(legitimate_data)
lof_scores = -lof.score_samples(X_all)

# Normalize scores
iso_scores_norm = (iso_scores - iso_scores.min()) / (iso_scores.max() - iso_scores.min())
svm_scores_norm = (svm_scores - svm_scores.min()) / (svm_scores.max() - svm_scores.min())
lof_scores_norm = (lof_scores - lof_scores.min()) / (lof_scores.max() - lof_scores.min())

# Ensemble (average)
ensemble_scores = (iso_scores_norm + svm_scores_norm + lof_scores_norm) / 3

# Compare
methods = {
    'Isolation Forest': iso_scores,
    'One-Class SVM': svm_scores,
    'Local Outlier Factor': lof_scores,
    'Ensemble': ensemble_scores
}

print("\n" + "="*60)
print("ANOMALY DETECTION METHODS COMPARISON")
print("="*60)

for name, scores in methods.items():
    auc = roc_auc_score(y_true, scores)
    print(f"{name:.<30} AUC: {auc:.3f}")

## 9. Visualizing Anomaly Scores

In [ ]:
# Plot score distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (name, scores) in enumerate(methods.items()):
    ax = axes[idx]
    
    # Plot histograms for each class
    ax.hist(scores[y_true == 0], bins=50, alpha=0.6, label='Legitimate', color='blue')
    ax.hist(scores[y_true == 1], bins=20, alpha=0.8, label='Fraud', color='red')
    ax.set_xlabel('Anomaly Score', fontsize=11)
    ax.set_ylabel('Count', fontsize=11)
    ax.set_title(f'{name} (AUC={roc_auc_score(y_true, scores):.3f})', fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  • Isolation Forest: Good separation between classes")
print("  • Ensemble: Best performance by combining multiple methods")
print("  • Overlap: Some fraud transactions have low anomaly scores")

## 10. Summary

### Anomaly Detection Methods:

1. **Isolation Forest**: 
   - Random partitioning, anomalies isolated quickly
   - Fast, scalable, works well with high-dimensional data

2. **One-Class SVM**:
   - Learns decision boundary around normal data
   - Slower, sensitive to kernel choice

3. **Local Outlier Factor**:
   - Density-based, compares to neighbors
   - Good for local anomalies

4. **Ensemble**:
   - Combines multiple methods
   - More robust, better performance

### Key Takeaways:

- ✅ **Unsupervised**: No fraud labels needed for training
- ✅ **Train on legitimate**: Model learns "normal" behavior
- ✅ **High anomaly score = suspicious**: Score indicates fraud likelihood
- ✅ **Ensemble works best**: Combine multiple detectors

### When to Use Anomaly Detection:

1. **Limited labeled fraud**: Few fraud examples available
2. **New fraud types**: Unknown fraud patterns
3. **Baseline detector**: Before deploying supervised models
4. **Ensemble component**: Combine with supervised methods

### Limitations:

- ❌ Lower precision than supervised methods
- ❌ Higher false positive rate
- ❌ Requires tuning contamination parameter

### Next Steps:
→ **Day 7**: Model Explainability (SHAP, LIME)
→ **Day 17**: Byzantine-Robust Aggregation (FL defenses)

---

**📁 Project Location**: `01_fraud_detection_core/anomaly_detection_benchmark/`